# McWhorter Problem
<cite>[McWhorter and Sunada][1]</cite> propose an analytical solution to the two-phase flow equation. A one-dimensional problem was considered which describes the flow of two incompressible, immiscible fluids through a porous medium, where the wetting phase (water) displaces the non-wetting fluid (air or oil) in the horizontal direction (without the influence of gravity).

![mcWhorter_concept_schematic.png](figures/mcWhorter_concept.png)

[1]: https://doi.org/10.1029/WR026i003p00399

### Material and Problem parameter

The parameter for this problem are stored in a CSV file. Let's look at the table below. The column "Key" shows the parameter name in the PRJ file. The column "Value" shows current parameter value. The columns "Range min", "Range max" and "Scale" are used in the next section to define sliders to interact with the analytic solution.

In [1]:
from helper import render_latex_table
import pandas as pd

df = pd.read_csv("parameter.csv", quotechar="'")

# We look at this column later
df_OGSTools = df[["Key", "OGSTools"]]
df.drop('OGSTools', axis=1, inplace=True)

render_latex_table(df, latex_column="Symbol")

Property,Symbol,Key,Value,Range min,Range max,Scale,Unit
Porosity,$$ \varphi $$,phi,0.15,0.145,0.175,linear,1
Intrinsic permeability,$$ K $$,K,1.0000e-09,5.0000e-10,5.0000e-09,log,m²
Residual saturation of the wetting phase,"$$ S_\mathrm{w,r} $$",Swr,0.1,0.001,0.2,linear,1
Residual saturation of the non-wetting phase,"$$ S_\mathrm{n,r} $$",Snr,0.001,0.0001,0.002,linear,1
Dynamic viscosity of the wetting phase,$$ \mu_\mathrm{n} $$,mun,0.001,0.0008,0.0012,log,Pa·s
Dynamic viscosity of the non-wetting pha,$$ \mu_\mathrm{w} $$,muw,2.0000e-05,1.0000e-05,3.0000e-05,log,Pa·s
Van Genuchten model parameter: entry pressue,$$ P_b $$,Pb,1.0000e+07,5.0000e+06,2.0000e+07,log,Pa
Van Genuchten model parameter: exponent,$$ m $$,exp,0.45,0.4,0.5,log,1
Initial saturation,$$ S_\mathrm{w}(t=0) $$,Si,0.9,0.95,0.85,linear,1
Injection boundary saturation,$$ S_\mathrm{w}(x=0) $$,S0,0.4,0.3,0.5,linear,1


<IPython.core.display.Javascript object>

In [2]:
!pip install pyDOE3

from pyDOE3 import pbdesign
import numpy as np

num_parameter = df.shape[0]
design = pbdesign(num_parameter)

num_simulation = np.shape(design)[0]

parameter = df["Key"]
value_min = df["Range min"]
value_max = df["Range max"]

print(design)


print(f"\nThere are {num_parameter} parameters and {num_simulation} simulation runs.") 

[[ 1. -1.  1.  1.  1. -1. -1. -1.  1. -1.]
 [-1.  1.  1.  1. -1. -1. -1.  1. -1. -1.]
 [ 1.  1.  1. -1. -1. -1.  1. -1. -1.  1.]
 [ 1.  1. -1. -1. -1.  1. -1. -1.  1. -1.]
 [ 1. -1. -1. -1.  1. -1. -1.  1. -1.  1.]
 [-1. -1. -1.  1. -1. -1.  1. -1.  1.  1.]
 [-1. -1.  1. -1. -1.  1. -1.  1.  1.  1.]
 [-1.  1. -1. -1.  1. -1.  1.  1.  1. -1.]
 [ 1. -1. -1.  1. -1.  1.  1.  1. -1. -1.]
 [-1. -1.  1. -1.  1.  1.  1. -1. -1. -1.]
 [-1.  1. -1.  1.  1.  1. -1. -1. -1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]

There are 10 parameters and 12 simulation runs.


In [3]:
import json

replace_info = dict()

for index, row in df_OGSTools.iterrows():
    if row["OGSTools"] == "Parameter":
        replace_info[row["Key"]] = row["OGSTools"]
    else:
        replace_info[row["Key"]] = json.loads(row["OGSTools"])

# Add infos for calculated parameter
replace_info["pci"] = "Parameter"
replace_info["pc0"] = "Parameter"

## Numerical solutions

For the numerical solution, we compare the Thermal-2-Phase-Hydro-Mechanical (TH2M) and the Two-phase Flow formulation.

1. The first step is to create a matching mesh that capture the penetration depth of the wetting fluid. 
2. Because the OGS models use the capillary pressure as a primary variable, the boundary and initial conditions need to be converted from saturations to pressures.

Tip: Pick the parameters you would like to study with the sliders above and run the code blocks below to compare the results of the analytical and numerical model.

In [4]:
from template import prj_from_template
from helper import create_1d_mesh, create_boundary_line_meshes

# Mesh for TwoPhaseFlowPP

depth = 0.5      # penetration depth of the wetting fluid (estimated!)
print(f"penetration depth of the wetting fluid: {depth}")

factor = 5.0   # mesh should be factor times larger than the depth 
Nel = 100         # number of mesh elements
mesh = create_1d_mesh(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1, mesh_type="line")
mesh.save(r"mesh_line.vtu")

coords = {'x1': mesh.GetBounds()[0], 'x2': 0.5*mesh.GetBounds()[0]}
prj_from_template(coords, "mcwt.template", "mcwt.gml")


    # 1.3 Mesh for TH2M
    # mesh = create_1d_mesh(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1, mesh_type="quad")
    # mesh.save(r"mesh_quad.vtu")

    # boundary_left, boundary_right = create_boundary_line_meshes(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1)
    # boundary_left.save("boundary_left_test.vtu")                                   

    # Match element numbers in boundary mesh to domain mesh
    # !identifySubdomains -m mesh_quad.vtu -- boundary_left_test.vtu

penetration depth of the wetting fluid: 0.5
2 replacements made.


In [ ]:
import pyvista as pv
from mcwhorter import VanGenuchten
import ogstools as ogs

import matplotlib.pyplot as plt

#labels = ["TwoPhaseFlowPP", "TH2M"]
labels = ["TwoPhaseFlowPP"]
results = {l: [] for l in labels}

for sim in np.arange(0, num_simulation):

    params = {k: vmin if d < 0 else vmax for d, k, vmin, vmax in zip(design[sim,:], parameter, value_min, value_max)}
    print(params)

    # 2. Calculate capillary pressures  
    model = VanGenuchten(1/params["Pb"], 1/(1-params["exp"]), params["Snr"], params["Swr"])
    params["pci"] = model.pc(params["Si"])
    params["pc0"] = model.pc(params["S0"])

    print(f"Si = {params['Si']} -> pci = {params['pci']}")
    print(f"S0 = {params['S0']} -> pc0 = {params['pc0']}")

    # 3. Updating the PRJ file and run simulations
    #prj_files = [r"mcWhorter_TwoPhasePP.prj", r"mcWhorter_TH2M.prj"]
    prj_files = [r"mcWhorter_TwoPhasePP.prj"]

    for prj_file in prj_files:
        prj = ogs.Project(input_file=prj_file, output_file=prj_file)
        for key in params.keys():
            try:
                if replace_info[key] == "Parameter":
                    prj.replace_parameter_value(name=key, value=params[key])
                else:
                    for r in replace_info[key]:
                        prj.replace_medium_property_value(mediumid=r["medium id"],
                                                        name=r["name"],
                                                        value=params[key],
                                                        propertytype=r["propertytype"],
                                                        valuetag=r["valuetag"])
            except:
                print(f"Parameter {key} not found...")
        prj.write_input()        
        prj.run_model()
    
    # 4. Evaluate results
    #resultfiles = ["twophaseflow_test_t_1000.000000.vtu", "result_McWhorter_H2_ts_110_t_1000.000000.vtu"]
    #resultfiles = ["twophaseflow_test_t_1000.000000.vtu"]
    resultfiles = ["twophaseflow_test_t_10000000.000000.vtu"]

    for label, resultfile in zip(labels, resultfiles):

        mesh = pv.read(resultfile)
        Sw=mesh["saturation"]
        x=mesh.points[:,0]

        ind = np.argsort(x)
        x = x[ind]
        Sw = Sw[ind]

        plt.plot(x, Sw)

        Swn = (Sw-min(Sw))/(max(Sw) - min(Sw))
        m = np.sum(x * Swn) / np.sum(Swn) # center of mass ~ depth
        print(m)

        #results[label].append(np.linalg.norm(Sw - np.interp(x, x_ref, Sw_ref)))
        results[label].append(m) #dummy value

    plt.plot()

ModuleNotFoundError: No module named 'ogstools'

Finally, we can update the PRJ file and run the simulations.

## Evaluation and Results

In [ ]:
import matplotlib.pyplot as plt

def calculate_effects_and_stddev(design, results):
    results = np.array(results)
    num_factors = design.shape[1]
    effects = []
    stddevs = []
    for i in range(num_factors):
        high = results[design[:, i] == 1]
        low = results[design[:, i] == -1]
        effect = (high.mean() - low.mean())  # Effekt
        stddev = np.sqrt(np.var(high) + np.var(low))  # Standardabweichung
        effects.append(effect)
        stddevs.append(stddev)
    return effects, stddevs

def plot_effects_with_errorbars(effects, stddevs, labels, title):
   
    # sort parameters according to effect size
    sorted_indices = np.argsort(np.abs(effects))[::-1]
    sorted_effects = np.array(effects)[sorted_indices]
    sorted_stddevs = np.array(stddevs)[sorted_indices]
    sorted_labels = np.array(labels)[sorted_indices]
    
    # plot with error bars
    plt.bar(sorted_labels, sorted_effects, yerr=sorted_stddevs, capsize=5, color="blue", alpha=0.7)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.title(title)
    plt.ylabel("effect size")
    plt.xlabel("parameter")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for l in labels:
    effects, stddevs = calculate_effects_and_stddev(design, results[l])
    plot_effects_with_errorbars(effects, stddevs, labels=parameter, title=l)
    plt.show()

## Literature

- McWhorter, D. B., and D. K. Sunada (1990), Exact integral solutions for two-phase flow, Water Resour. Res., 26(3), 399–413, <cite>[doi:10.1029/WR026i003p00399][1]</cite>. 
- Radek Fučík, https://mmg.fjfi.cvut.cz/~fucik/index.php?page=exact

[1]: https://doi.org/10.1029/WR026i003p00399
